In [1]:
import numpy as np
import tensorflow as tf

# Notation

Convolution is an operation between matrices or tensors denoted by $*$.


| Symbols                   | Meaning               | Size                                              |
|:--------------------------|:----------------------|:--------------------------------------------------|
| $\boldsymbol{X}$          | 2D Input Matrix       | $(H_{in}$, $W_{in})$                              |
| $\boldsymbol{\mathsf{X}}$ | 3D Input Tensor       | $(H_{in}$, $W_{in}$, $C_{in})$                    |
| $\boldsymbol{\mathsf{X}}$ | 4D Input Tensor       | $(N_{in}$, $H_{in}$, $W_{in}$, $C_{in})$          |
| $\boldsymbol{W}$          | 2D Filter Matrix      | $(H_{filter}$, $W_{filter})$                      |
| $\boldsymbol{\mathsf{W}}$ | 3D Filter Tensor      | $(H_{filter}$, $W_{filter}$, $C_{in})$            |
| $\boldsymbol{\mathsf{W}}$ | 4D Filter Tensor      | $(H_{filter}$, $W_{filter}$, $C_{in}$, $C_{out})$ |
| $\boldsymbol{Y}$          | 2D Output Matrix      | $(H_{out}$, $W_{out})$                            |
| $\boldsymbol{\mathsf{Y}}$ | 3D Output Tensor      | $(H_{out}$, $W_{out}$, $C_{out})$                 |
| $\boldsymbol{\mathsf{Y}}$ | 4D Output Tensor      | $(N_{in}$, $H_{out}$, $W_{out}$, $C_{out})$       |
 
Please note that letter `w` stands for `weight` as in $\boldsymbol{W}$ or $\boldsymbol{\mathsf{W}}$, and `width` as in $W_{in}$, too.

Also, a convolution can takes padding $P$ and stride $S$.

# Convolution Operation Forward Pass

Try calculation with NumPy and TensorFlow for the followings

1. Basic $(4 \times 4) * (3 \times 3) = (2 \times 2)$
2. Padding $(4 \times 4) * (3 \times 3) = (4 \times 4)$ where $P=1$
3. Stride $(7 \times 7) * (3 \times 3) = (3 \times 3)$ where $S=2$
4. Stride $(7 \times 7) * (3 \times 3) = (4 \times 4)$ where $P=1, S=2$
5. Channel (2x2) $*$ (2x2)
6. Channel and bias (2x2) $*$ (2x2) + (1)
7. Multiple Kernels (2x2) $*$ (2x2)
8. Multiple Kernels + bias (2x2) $*$ (3x3x3x4) + (1x1x4)
9. Mini-batch (2x4x4x3) $*$ (3x3x3x4)
10. All

In [27]:
def float_sequence(size):
    return np.arange(size, dtype=np.float32)

### 1. Basic Convolution

$(4 \times 4) * (3 \times 3) = (2 \times 2)$

In [41]:
X = float_sequence(4*4).reshape(4,4)
W = 12 - float_sequence(3*3).reshape(3,3)
print("=== X ===")
print(X)
print("=== W ===")
print(W)

Y = np.zeros((2,2))
for w in range(4-3+1):
    for h in range(4-3+1):
        for i in range(3):
            for j in range(3):
                Y[h,w] += X[h+i, w+j] * W[i, j]

print("=== Y ===")     
print(Y)

with tf.Session() as sess:
    Y_tf = sess.run(tf.nn.conv2d(
        X.reshape(1, 4, 4, 1),
        W.reshape(3, 3, 1, 1),
        strides=[1, 1, 1, 1],
        padding='VALID'
    ))
    print("=== Y (TF) ===")     
    print(Y_tf[0, :, :, 0])

print("=== Matched? ===")    
print(Y == Y_tf[0, :, :, 0])

=== X ===
[[  0.   1.   2.   3.]
 [  4.   5.   6.   7.]
 [  8.   9.  10.  11.]
 [ 12.  13.  14.  15.]]
=== W ===
[[ 12.  11.  10.]
 [  9.   8.   7.]
 [  6.   5.   4.]]
=== Y ===
[[ 282.  354.]
 [ 570.  642.]]
=== Y (TF) ===
[[ 282.  354.]
 [ 570.  642.]]
=== Matched? ===
[[ True  True]
 [ True  True]]


### 2. Convolution with padding

$(4 \times 4) * (3 \times 3) = (4 \times 4)$ where $P=1$

[numpy.pad()](https://docs.scipy.org/doc/numpy/reference/generated/numpy.pad.html)

In [50]:
X = float_sequence(4*4).reshape(4,4)
pad = 1
X = np.pad(X, ((pad, pad), (pad, pad)), 'constant')
W = 12 - float_sequence(3*3).reshape(3,3)
print("=== X ===")
print(X)
print("=== W ===")
print(W)

Y = np.zeros((4,4))
for w in range(6-3+1):
    for h in range(6-3+1):
        h_start = h
        h_end   = h_start + 3
        w_start = w
        w_end   = w_start + 3
        Y[h,w] = np.sum(X[h_start:h_end, w_start:w_end] * W)

print("=== Y ===")     
print(Y)

with tf.Session() as sess:
    Y_tf = sess.run(tf.nn.conv2d(
        X.reshape(1, 6, 6, 1),
        W.reshape(3, 3, 1, 1),
        strides=[1, 1, 1, 1],
        padding='VALID'
    ))
    print("=== Y (TF) ===")     
    print(Y_tf[0, :, :, 0])

print("=== Matched? ===")    
print(Y == Y_tf[0, :, :, 0])

=== X ===
[[  0.   0.   0.   0.   0.   0.]
 [  0.   0.   1.   2.   3.   0.]
 [  0.   4.   5.   6.   7.   0.]
 [  0.   8.   9.  10.  11.   0.]
 [  0.  12.  13.  14.  15.   0.]
 [  0.   0.   0.   0.   0.   0.]]
=== W ===
[[ 12.  11.  10.]
 [  9.   8.   7.]
 [  6.   5.   4.]]
=== Y ===
[[  47.   95.  134.  113.]
 [ 153.  282.  354.  282.]
 [ 333.  570.  642.  486.]
 [ 365.  605.  662.  487.]]
=== Y (TF) ===
[[  47.   95.  134.  113.]
 [ 153.  282.  354.  282.]
 [ 333.  570.  642.  486.]
 [ 365.  605.  662.  487.]]
=== Matched? ===
[[ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]]


### 3. Convolution with Stride

$(7 \times 7) * (3 \times 3) = (3 \times 3)$ where $S=2$

In [54]:
X = float_sequence(7*7).reshape(7,7)
W = 12 - float_sequence(3*3).reshape(3,3)
S = 2
print("=== X ===")
print(X)
print("=== W ===")
print(W)

Y = np.zeros((3,3))
for w in range((7-3)//2+1):
    for h in range((7-3)//2+1):
        h_start = h * S
        h_end   = h_start + 3
        w_start = w * S
        w_end   = w_start + 3
        Y[h,w] = np.sum(X[h_start:h_end, w_start:w_end] * W)

print("=== Y ===")     
print(Y)

with tf.Session() as sess:
    Y_tf = sess.run(tf.nn.conv2d(
        X.reshape(1, 7, 7, 1),
        W.reshape(3, 3, 1, 1),
        strides=[1, 2, 2, 1],
        padding='VALID'
    ))
    print("=== Y (TF) ===")     
    print(Y_tf[0, :, :, 0])

print("=== Matched? ===")    
print(Y == Y_tf[0, :, :, 0])

=== X ===
[[  0.   1.   2.   3.   4.   5.   6.]
 [  7.   8.   9.  10.  11.  12.  13.]
 [ 14.  15.  16.  17.  18.  19.  20.]
 [ 21.  22.  23.  24.  25.  26.  27.]
 [ 28.  29.  30.  31.  32.  33.  34.]
 [ 35.  36.  37.  38.  39.  40.  41.]
 [ 42.  43.  44.  45.  46.  47.  48.]]
=== W ===
[[ 12.  11.  10.]
 [  9.   8.   7.]
 [  6.   5.   4.]]
=== Y ===
[[  444.   588.   732.]
 [ 1452.  1596.  1740.]
 [ 2460.  2604.  2748.]]
=== Y (TF) ===
[[  444.   588.   732.]
 [ 1452.  1596.  1740.]
 [ 2460.  2604.  2748.]]
=== Matched? ===
[[ True  True  True]
 [ True  True  True]
 [ True  True  True]]


# Convolution Operation Naive Backward Pass

Try calculating NumPy, then check with TensorFlow and numerical gradient

# Fast Forward Pass with im2col

Try calculating NumPy, then check with the result above

# Convolution Operation Fast Backward

Try calculating NumPy, then check with TensorFlow

# Fast Forward Pass with im2col

Try calculating NumPy, then check with the result above